In [1]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import numpy
import PIL
from PIL import Image
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate


/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import scipy.io
mat = scipy.io.loadmat('/home/aniruddha/deep-learning-projects/Siamese_Networks/Dataset/PaviaCentre.mat')

In [3]:
arr = mat['pavia']
arr = np.array(arr)
print(arr.shape)

(1096, 715, 102)


In [4]:
print(arr[100][300])

[329 280 299 265 148 107  87 176 277 284 227 229 264 304 345 364 373 363
 342 364 412 436 409 407 425 434 445 452 454 464 478 481 468 494 523 530
 515 507 509 497 459 435 401 352 342 341 336 345 345 340 339 327 324 306
 283 284 287 272 261 260 257 233 227 235 230 217 221 217 191 171 169 172
 154 135 128 127 104  91 107 110  96  91 104 125 123  86  77  98 108  98
  98 100 111 106  75  80  99 111 114 133 127 100]


In [5]:
print(arr[100][3])
print(arr[100][2])

[ 762  799  610  534  550  581  593  566  604  632  661  628  621  652
  646  621  627  664  691  689  709  744  753  753  745  751  751  746
  767  805  827  849  878  919  959 1002 1049 1089 1126 1171 1214 1249
 1282 1325 1362 1382 1395 1402 1419 1411 1429 1469 1465 1472 1479 1497
 1508 1526 1527 1511 1530 1566 1585 1601 1599 1582 1626 1685 1719 1716
 1723 1762 1782 1762 1742 1765 1798 1803 1761 1736 1768 1848 1877 1781
 1728 1788 1829 1821 1809 1814 1836 1815 1792 1797 1832 1830 1796 1774
 1743 1726 1730 1751]
[1161 1130  871  774  794  747  681  657  631  692  747  788  744  681
  654  657  678  666  666  712  736  738  747  778  772  748  728  732
  762  817  879  893  892  902  935  989 1033 1067 1116 1174 1174 1189
 1250 1287 1307 1354 1382 1374 1366 1392 1411 1419 1432 1445 1449 1462
 1480 1492 1517 1542 1540 1539 1559 1574 1569 1567 1595 1616 1642 1653
 1673 1711 1740 1725 1702 1706 1725 1733 1764 1795 1824 1834 1824 1779
 1760 1782 1795 1807 1810 1805 1791 1758 1721 1748 1779

In [6]:
import scipy.io
mat = scipy.io.loadmat('/home/aniruddha/deep-learning-projects/Siamese_Networks/Dataset/PaviaCentre_gt.mat')

In [7]:
arr1 = mat['pavia_gt']
arr1 = np.array(arr1)
print(arr1.shape)

(1096, 715)


In [8]:
print(arr1[100])

[0 0 0 0 0 0 0 0 0 0 0 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 8 8 8 8 8 8 8 0 0 0 0 0 0 0 0 0 0 8 8 8 0 0 0 0 0 0 0 0 0
 8 8 0 0 0 0 0 0 0 0 0 0 0 0 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 6 6 6 0 0 0 0 0 0 9 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [10]:
print(arr1.max())

9


In [11]:
a=[]
label=[]
k=0
for i in range(0,arr1.shape[0]):
    for j in range(0,arr1[i].shape[0]):
        a.append(arr[i][j])
        label.append(arr1[i][j])
        
a=np.array(a)
label=np.array(label)
print(a.shape)
print(label.shape)

(783640, 102)
(783640,)


In [12]:
print(a[100])
print(label[100])

[ 619  436  419  553  671  829  900  875  944  905  892  816  751  788
  864  885  874  871  875  897  918  925  924  918  919  935  925  922
  921  918  888  845  845  885  917  949  950  945  948  951  950  944
  940  935  937  945  924  907  916  946  978  972  939  950  965  960
  963  965  965  956  955  956  952  953  959  957  946  924  943  957
  971  976  978  963  942  970  980  972  976  986 1000 1000 1007  993
  970  985 1001  987  976  972  973  993 1001  993  993  995  987  972
  944  925  920  947]
0


In [17]:
X_train=[]
y_train=[]
for i in range (0,a.shape[0]):
    if(label[i]==1):
        y_train.append(0)
    if(label[i]==2):
        y_train.append(1)
    if(label[i]==3):
        y_train.append(2)
    if(label[i]==4):
        y_train.append(3)
    if(label[i]==6):
        y_train.append(4)
    if(label[i]==7):
        y_train.append(5)
    if(label[i]==8):
        y_train.append(6)
    if (label[i]==1 or label[i]==2 or label[i]==3 or label[i]==4 or label[i]==6 or label[i]==7 or label[i]==8):
        X_train.append(a[i])
X_train=np.array(X_train)
y_train=np.array(y_train)
print(X_train.shape)
print(y_train.shape)
        

(138705, 102)
(138705,)


In [18]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state = 0)

In [19]:
import scipy.io
mat = scipy.io.loadmat('/home/aniruddha/deep-learning-projects/Siamese_Networks/Dataset/PaviaU.mat')

In [21]:
arr = mat['paviaU']
arr = np.array(arr)
print(arr.shape)

(610, 340, 103)


In [26]:
import scipy.io
mat = scipy.io.loadmat('/home/aniruddha/deep-learning-projects/Siamese_Networks/Dataset/PaviaU_gt.mat')

In [27]:
arr1 = mat['paviaU_gt']
arr1 = np.array(arr1)
print(arr1.shape)

(610, 340)


In [29]:
print(arr1[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 4 4 4 4 4 4 4 4 4 4 4 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]


In [30]:
a=[]
label=[]
k=0
for i in range(0,arr1.shape[0]):
    for j in range(0,arr1[i].shape[0]):
        a.append(arr[i][j])
        label.append(arr1[i][j])
        
a=np.array(a)
label=np.array(label)
print(a.shape)
print(label.shape)

(207400, 103)
(207400,)


In [31]:
X_train1=[]
y_train1=[]
for i in range (0,a.shape[0]):
    if(label[i]==3):
        y_train1.append(0)
    if(label[i]==0):
        y_train1.append(1)
    if(label[i]==7):
        y_train1.append(2)
    if(label[i]==6):
        y_train1.append(3)
    if(label[i]==8):
        y_train1.append(4)
    if(label[i]==1):
        y_train1.append(5)
    if(label[i]==5):
        y_train1.append(6)
    if (label[i]==3 or label[i]==0 or label[i]==7 or label[i]==6 or label[i]==8 or label[i]==1 or label[i]==5):
        X_train1.append(a[i])
X_train1=np.array(X_train1)
y_train1=np.array(y_train1)
print(X_train1.shape)
print(y_train1.shape)
        

(184740, 103)
(184740,)


In [32]:
from sklearn.utils import shuffle
X_train1, y_train1 = shuffle(X_train1, y_train1, random_state = 0)

In [38]:
print(X_train[100])
print(X_train[10000])
print(y_train[100])
print(y_train[10000])

[  0   0  30 154 240 228 215 152 228 300 288 278 279 279 280 279 315 332
 342 336 310 308 342 359 355 381 417 427 440 453 454 464 468 452 452 469
 457 436 438 439 415 376 327 287 263 255 239 248 270 263 244 234 227 224
 220 224 205 185 172 159 158 182 188 165 153 155 134 116 101  75  74  73
  54  27  15  29  40  25  13  12  10   5   0   0  27  41  32  15   7  23
  31  24  23  28  24   0   0   0   2  24  41  23]
[ 802  607  669  821  846  847  840  698  586  649  745  778  750  785
  832  832  840  876  912  908  886  865  866  888  942  984  990 1004
 1036 1060 1061 1075 1090 1115 1146 1171 1208 1279 1344 1362 1377 1403
 1452 1492 1515 1536 1545 1540 1554 1561 1582 1611 1661 1672 1651 1634
 1635 1642 1681 1722 1747 1761 1747 1750 1759 1743 1781 1810 1828 1859
 1886 1907 1876 1847 1885 1912 1909 1909 1930 1948 1959 1982 1986 1912
 1859 1883 1918 1926 1927 1927 1918 1899 1919 1938 1929 1913 1882 1858
 1843 1815 1817 1836]
0
6


In [41]:
from sklearn.preprocessing import StandardScaler
x=X_train1
x = StandardScaler().fit_transform(x)
print(x[100])

/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


[-0.5987657  -0.20141909 -0.37718137 -0.58349374 -0.75344376 -0.99558269
 -1.07248813 -1.01551233 -0.97702595 -0.84565065 -0.81867816 -0.8756142
 -0.92063077 -0.91692369 -0.88100695 -0.90641084 -0.90521145 -0.90105895
 -0.92715977 -0.99039067 -1.06009572 -1.09559987 -1.10127383 -1.11451962
 -1.13691611 -1.1718671  -1.21738531 -1.26067795 -1.29694883 -1.33617316
 -1.36550714 -1.37642463 -1.40265046 -1.42167376 -1.4057887  -1.4005116
 -1.41871045 -1.4062724  -1.39067754 -1.39386309 -1.37976323 -1.36999027
 -1.38325579 -1.38181893 -1.37957934 -1.38985038 -1.38637825 -1.38261497
 -1.38558361 -1.41521377 -1.42260509 -1.38452591 -1.36031597 -1.34980058
 -1.34441102 -1.35057803 -1.3408363  -1.34153001 -1.33055725 -1.32318237
 -1.31944475 -1.33939308 -1.36046443 -1.35086011 -1.35265308 -1.38701327
 -1.4287243  -1.45958998 -1.51234656 -1.58490144 -1.6404652  -1.69544847
 -1.70403083 -1.65683253 -1.58143855 -1.46103341 -1.36896448 -1.27897936
 -1.20431145 -1.15539978 -1.1168379  -1.09518327 -1.0

In [42]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
principalComponents = pca.fit_transform(x)
print(principalComponents.shape)


(184740, 50)


In [43]:
x=principalComponents
print(x[100])

[-1.13670500e+01 -2.75604667e+00  1.50038577e+00  8.09963891e-01
  5.71875684e-01 -4.06355956e-01 -6.20082616e-01  6.22962574e-01
  6.99515623e-02 -7.01710789e-02 -1.65370110e-01 -1.50507030e-01
 -1.85494846e-01  1.75634413e-01  9.52377280e-02 -7.78752333e-02
  6.96115934e-02  5.50198988e-03 -4.19637720e-03  3.08792296e-02
  4.49275545e-02 -2.01565423e-02  7.80304063e-02 -1.18887050e-01
 -1.29893489e-02  4.23406651e-03 -1.26645111e-02 -1.16906591e-02
  1.52452897e-02  5.35971595e-02 -2.80558012e-03  3.84323549e-02
 -1.48172303e-02  4.51111453e-03 -2.99352766e-02 -1.41979731e-02
 -1.42121738e-02  2.06523455e-02 -5.65173058e-02  4.49130498e-02
 -3.37018207e-02  2.51853978e-02  1.56719846e-02 -4.01579371e-03
  6.73870660e-03 -2.33660636e-02 -1.26052267e-02  3.23707942e-03
  1.02133865e-02  1.08656364e-02]
